In [17]:
import ast
import jedi

project_dir = "src"
file_name = "jedi_test.py"

file_contents = open(f"{project_dir}/{file_name}").read()

project = jedi.Project(path=project_dir)
script = jedi.Script(file_contents, path=file_name, project=project)
tree = ast.parse(file_contents)

for node in ast.walk(tree):
    if node.__class__.__name__ == 'Call':
        function_definition = script.get_signatures(
            node.lineno, 
            # node.col_offset + 2,
            19
        )
        function_definitions = script.goto(
            node.lineno, 
            node.col_offset,
            follow_imports=True,
            follow_builtin_imports=True,
        )
        print(node.lineno, node.col_offset)
        if function_definitions:
            function_definition = function_definitions[0]
            function_definition

for function in script.get_names():
    pass
#     if function.type == 'function':
#         print(function)
        # definition = function.goto_assignments()
        # if definition:
        #     print(f"{function.name} is defined at {definition[0].module_path}:{definition[0].line}")

4 11
<Name full_name='helpers.test2.a_func', description='def a_func'>
